In [10]:
log_names = ['27_03_2021_21_13_38_task_standard_seed_7', '27_03_2021_21_18_39_task_standard_seed_122', '27_03_2021_21_22_35_task_standard_seed_809', '27_03_2021_21_26_32_task_standard_seed_7', '27_03_2021_21_37_58_task_standard_seed_122', '27_03_2021_21_46_57_task_standard_seed_809', '27_03_2021_21_58_31_task_standard_seed_7', '27_03_2021_22_10_58_task_standard_seed_122', '27_03_2021_22_23_37_task_standard_seed_809', '27_03_2021_22_35_48_task_img_clas_seed_7', '27_03_2021_22_40_54_task_img_clas_seed_122', '27_03_2021_22_52_54_task_img_clas_seed_809', '27_03_2021_22_57_58_task_img_clas_seed_7', '27_03_2021_23_12_40_task_img_clas_seed_122', '27_03_2021_23_28_27_task_img_clas_seed_809', '27_03_2021_23_43_42_task_img_clas_seed_7', '27_03_2021_23_59_23_task_img_clas_seed_122', '28_03_2021_00_13_58_task_img_clas_seed_809', '28_03_2021_00_58_18_task_target_clas_seed_7', '28_03_2021_01_04_29_task_target_clas_seed_122', '28_03_2021_01_09_23_task_target_clas_seed_809', '28_03_2021_01_23_40_task_target_clas_seed_7', '28_03_2021_01_37_44_task_target_clas_seed_122', '28_03_2021_01_52_23_task_target_clas_seed_809', '28_03_2021_03_06_28_task_target_clas_seed_7', '28_03_2021_03_20_31_task_target_clas_seed_122', '28_03_2021_03_34_47_task_target_clas_seed_809']

In [11]:
from tensorboard.backend.event_processing import event_accumulator
import os
import numpy as np
import pandas as pd
import json
import torch
pd.options.display.float_format = '{:,.3f}'.format

def load_metric_from_event(event_name): 
    event_path = os.path.join("runs", event_name)
    event_name = os.listdir(event_path)[0]
    event_path = os.path.join(event_path, event_name)

    # only read scalars
    ea = event_accumulator.EventAccumulator(event_path, size_guidance={event_accumulator.SCALARS: 0})

    # load scalars
    ea.Reload()

    return get_best_epoch_metrics(ea)

def get_best_epoch_metrics(ea):
    test_accs = ea.Scalars(f"test/acc")
    test_acc_vals = [acc.value for acc in test_accs]
    test_best_acc_idx = np.argmax(test_acc_vals)
    test_best_acc = np.max(test_acc_vals)
    test_best_epoch = test_accs[test_best_acc_idx].step

    train_accs = ea.Scalars(f"train/acc")
    train_best_acc = train_accs[test_best_acc_idx].value
    try:
        train_img_clas_accs = ea.Scalars(f"train/img_class_acc")
        train_best_img_clas_acc = train_img_clas_accs[test_best_acc_idx].value

        
        test_img_clas_accs = ea.Scalars(f"train/img_class_acc")
        test_best_img_clas_acc = test_img_clas_accs[test_best_acc_idx].value
    except KeyError:
        train_best_img_clas_acc = np.nan
        test_best_img_clas_acc = np.nan
    
    return [test_best_epoch, train_best_acc, test_best_acc, train_best_img_clas_acc, test_best_img_clas_acc]

## Define important variables

In [12]:
# events in tensorboard


# choose which columsn to include
include_params = ["task", "ic_loss_weight", "num_imgs", "same_class_prob", "seed"]

In [18]:
results = []
for event_name in log_names:
    # get parameters
    with open(f"args//args_{event_name}.json") as json_file:
        params = json.load(json_file)
    # wrap each value into a list
    params = {key:[value] for key, value in params.items()}

    # create dataframe and add parameters
    results_df = pd.DataFrame(params)[include_params]

    # add metrics
    metric_cols = ["best epoch", "best train acc", "best test acc", "best train img acc", "best test img acc"]
    results_df.loc[:, metric_cols] = load_metric_from_event(event_name)

    # add noise acc
    try:
        path = f'interactions//{event_name}//validation'
        interactions = torch.load(f"{path}//interactions_epoch11")
        results_df["noise acc"] = interactions.aux["acc"].mean().item()
    except FileNotFoundError:
        results_df["noise acc"] = 0.0

    # append to final table
    results.append(results_df)


results = pd.concat(results)

results[["task", "same_class_prob", "seed", "best test acc", "noise acc"]]

,task,same_class_prob,seed,best test acc,noise acc
0,standard,0.000,7,0.978,0.644
0,standard,0.000,122,0.978,0.766
0,standard,0.000,809,0.973,0.795
0,standard,0.500,7,0.954,0.824
0,standard,0.500,122,0.970,0.734
0,standard,0.500,809,0.942,0.869
0,standard,1.000,7,0.961,0.783
0,standard,1.000,122,0.939,0.859
0,standard,1.000,809,0.936,0.827
0,img_clas,0.000,7,0.974,0.630


In [27]:
from scipy.stats import pearsonr, spearmanr
pearson = pearsonr(results["best test acc"], results["noise acc"])
spearman = spearmanr(results["best test acc"], results["noise acc"])
print(f"Pearson correlation: {pearson[0]} (p={np.round(pearson[1],2)})")
print(f"Spearman correlation: {spearman.correlation} (p={np.round(spearman.pvalue, 2)})")

Pearson correlation: 0.08301405852828952 (p=0.68)
Spearman correlation: 0.01282051282051282 (p=0.95)


In [14]:
# results["success"] = (results["best test acc"]>0.97).astype(int)
results = results.groupby(by=["task", "same_class_prob"]).mean()
results = results.reindex(index = ['standard','img_clas','target_clas'], level="task").fillna('-')
results[["best test acc", "best test img acc"]]

best test acc best test img acc
task        same_class_prob                                 
standard    0.000                    0.976                 -
            0.500                    0.955                 -
            1.000                    0.945                 -
img_clas    0.000                    0.973             0.918
            0.500                    0.923             0.856
            1.000                    0.935             1.000
target_clas 0.000                    0.972             0.379
            0.500                    0.918             0.455
            1.000                    0.905             0.414

In [15]:
noise_results = results.reset_index(level="same_class_prob")
noise_results = noise_results.loc[:,["best test acc", "noise acc"]]
noise_results

,best test acc,noise acc
task,,
standard,0.976,0.735
standard,0.955,0.809
standard,0.945,0.823
img_clas,0.973,0.703
img_clas,0.923,0.746
img_clas,0.935,0.845
target_clas,0.972,0.600
target_clas,0.918,0.594
target_clas,0.905,0.688


In [134]:
# success_results = results[results["success"]==1].groupby(by=["task", "same_class_prob"])["best epoch"].mean()

In [136]:
results = results.drop(["ic_loss_weight", "num_imgs", "best epoch", "best train acc", "best train img acc"], axis=1)
results = results.groupby(by=["task", "same_class_prob"]).mean().drop(["seed"], axis=1) #.agg({'best test acc' : ['mean'], 'best test img acc':['mean'], "success":['mean']}) 

In [137]:
results = results.reindex(index = ['standard','img_clas','target_clas'], level="task").fillna('-')

In [138]:
pd.options.display.float_format = '{:,.3f}'.format

In [139]:
results = results.drop(["success"], axis=1)

In [140]:
results

best test acc best test img acc
task        same_class_prob                                 
standard    0.000                    0.974                 -
            0.500                    0.947                 -
            1.000                    0.941                 -
img_clas    0.000                    0.964             0.909
            0.500                    0.958             0.887
            1.000                    0.937             1.000
target_clas 0.000                    0.975             0.339
            0.500                    0.919             0.434
            1.000                    0.917             0.505

In [123]:
print(results.to_latex())

\begin{tabular}{llrrl}
\toprule
            &       &  best epoch &  best test acc & best test img acc \\
task & same\_class\_prob &             &                &                   \\
\midrule
standard & 0.000 &       2.667 &          0.974 &                 - \\
            & 0.500 &       6.667 &          0.947 &                 - \\
            & 1.000 &      10.667 &          0.941 &                 - \\
img\_clas & 0.000 &       5.000 &          0.964 &             0.909 \\
            & 0.500 &       8.667 &          0.958 &             0.887 \\
            & 1.000 &       9.667 &          0.937 &             1.000 \\
target\_clas & 0.000 &       3.000 &          0.975 &             0.339 \\
            & 0.500 &       6.667 &          0.919 &             0.434 \\
            & 1.000 &       8.667 &          0.917 &             0.505 \\
\bottomrule
\end{tabular}



In [ ]:
# events = ['24_03_2021_10_33_16_task_standard_seed_7', '24_03_2021_10_37_04_task_standard_seed_122', '24_03_2021_10_39_50_task_standard_seed_809', '24_03_2021_10_43_50_task_standard_seed_7', '24_03_2021_11_00_48_task_standard_seed_122', '24_03_2021_11_18_26_task_standard_seed_809', '24_03_2021_11_35_57_task_standard_seed_7', '24_03_2021_11_54_58_task_standard_seed_122', '24_03_2021_12_13_03_task_standard_seed_809', '24_03_2021_12_30_12_task_img_clas_seed_7', '24_03_2021_12_37_57_task_img_clas_seed_122', '24_03_2021_12_44_03_task_img_clas_seed_809', '24_03_2021_13_05_47_task_img_clas_seed_7', '24_03_2021_13_27_19_task_img_clas_seed_122', '24_03_2021_13_48_31_task_img_clas_seed_809', '24_03_2021_13_59_50_task_img_clas_seed_7', '24_03_2021_14_21_20_task_img_clas_seed_122', '24_03_2021_14_43_14_task_img_clas_seed_809', '24_03_2021_15_05_02_task_target_clas_seed_7', '24_03_2021_15_09_49_task_target_clas_seed_122', '24_03_2021_15_14_35_task_target_clas_seed_809', '24_03_2021_15_19_17_task_target_clas_seed_7', '24_03_2021_15_41_05_task_target_clas_seed_122', '24_03_2021_16_02_59_task_target_clas_seed_809', '24_03_2021_16_24_57_task_target_clas_seed_7', '24_03_2021_16_47_18_task_target_clas_seed_122', '24_03_2021_17_09_55_task_target_clas_seed_809']